In [1]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np 
import keras

In [3]:
train = c.data.make_base_df()
train = c.processing.transform_save_imgs(train, c.processing.resize_n, n=224); train.head()


,path,Type,filetype,num_id,processed_path
0,../data/train/Type_1/1324.jpg,1,jpg,1324,../data/processed/resize_224/train/1324.jpg
1,../data/train/Type_1/583.jpg,1,jpg,583,../data/processed/resize_224/train/583.jpg
2,../data/train/Type_1/551.jpg,1,jpg,551,../data/processed/resize_224/train/551.jpg
3,../data/train/Type_1/229.jpg,1,jpg,229,../data/processed/resize_224/train/229.jpg
4,../data/train/Type_1/1390.jpg,1,jpg,1390,../data/processed/resize_224/train/1390.jpg


In [4]:
test = c.data.make_test_df()
test = c.processing.transform_save_imgs(test, c.processing.resize_n, n=224)

10254 11554 11967 10460 12605 12888 13219 11215 10533 11037 10466 11056 10475 13490 12009 11633 11585 10978 12654 10148 13416 11990 

KeyboardInterrupt: 

In [5]:
model_fp = '../model/vgg_4blocks_fullytrained.hk'
model = keras.models.load_model(model_fp)

In [6]:
import cv2
def df_to_keras_generator(df, batch_size, grayscale=None):
    assert isinstance(grayscale, (bool)), 'grayscale must be set to a bool'
    imread_opt = 0 if grayscale else 1 # 1 is 3chan rbg, 0 is grayscale

    batch_features = np.zeros((batch_size, 224, 224, 3))

    batch_counter = 0
    while 1:
        for index, row in df.iterrows():
            img = cv2.imread(row['path'], imread_opt)
            batch_features[batch_counter] = cv2.resize(img, (224, 224), cv2.INTER_LINEAR)
            batch_counter += 1
            if batch_counter == batch_size:
                batch_counter = 0
                yield batch_features


In [ ]:
batch_size_t = 10
predictions = model.predict_generator(df_to_keras_generator(test, batch_size_t, grayscale=False)
                    ,len(test))

In [ ]:
predictions

In [ ]:
test = c.processing.append_probabilities(test, predictions, ['1','2','3']); 

In [ ]:
test.head()

In [ ]:
c.submission.write_submission_file('output_vgg_4blocks_fullytrained.csv', test)